
# CHECKPOINT 01 – Data Science e Machine Learning (Python & Orange)

**Data:** 2025-09-02

Este notebook resolve e/ou guia a resolução dos **40 exercícios** com base nos datasets da UCI:
- *Individual Household Electric Power Consumption*
- *Appliances Energy Prediction*

> **Antes de começar**: coloque os arquivos em `../data/`:
> - `household_power_consumption.txt`
> - `energydata_complete.csv`


In [ ]:

# Instalação rápida (se necessário)
# %pip install -q -r ../requirements.txt
import warnings; warnings.filterwarnings("ignore")
import pandas as pd, numpy as np, matplotlib.pyplot as plt
from pathlib import Path
from src.utils import optimize_power_dtypes, add_datetime, add_weekday_column, minmax_scale
DATA_DIR = Path('../data')
POWER_PATH = DATA_DIR / 'household_power_consumption.txt'
APPLIANCES_PATH = DATA_DIR / 'energydata_complete.csv'
pd.set_option("display.max_rows", 10)


## PARTE 1 – Exercícios iniciais: *Individual Household Electric Power Consumption*

### 1) Carregue o dataset e exiba as 10 primeiras linhas.

In [ ]:

# O arquivo é separado por ';' e usa '?' como missing.
usecols = ['Date','Time','Global_active_power','Global_reactive_power','Voltage',
           'Global_intensity','Sub_metering_1','Sub_metering_2','Sub_metering_3']
dtypes = {c:'float32' for c in usecols if c not in ['Date','Time']}
dtypes.update({'Date':'string','Time':'string'})
power = pd.read_csv(POWER_PATH, sep=';', na_values='?', usecols=usecols, dtype=dtypes, low_memory=True)
power = optimize_power_dtypes(power)
power.head(10)


### 2) Diferença entre **Global_active_power** e **Global_reactive_power**


- **Global_active_power**: potência **ativa** (kW) efetivamente convertida em trabalho/energia útil.
- **Global_reactive_power**: potência **reativa** (kVar) associada a campos magnéticos/capacitivos — não realiza trabalho útil, mas circula entre fonte e carga, afetando fator de potência.


### 3) Valores ausentes: verifique e quantifique.

In [ ]:

power.isna().sum().to_frame('missing')


### 4) Converter `Date` para datetime e criar `weekday`.

In [ ]:

power_dt = power.copy()
power_dt['Date_dt'] = pd.to_datetime(power_dt['Date'], format='%d/%m/%Y', errors='coerce')
power_dt['weekday'] = power_dt['Date_dt'].dt.day_name()
power_dt[['Date','Date_dt','weekday']].head()


### 5) Filtrar 2007 e média diária de `Global_active_power`.

In [ ]:

mask2007 = power_dt['Date_dt'].dt.year == 2007
daily_mean_2007 = (power_dt.loc[mask2007]
                   .groupby(power_dt['Date_dt'].dt.date)['Global_active_power']
                   .mean())
daily_mean_2007.head(), daily_mean_2007.mean()


### 6) Gráfico de linha de `Global_active_power` em um dia.

In [ ]:

# Escolha de exemplo: 2007-01-15 (altere se desejar)
day = pd.Timestamp('2007-01-15')
day_mask = power_dt['Date_dt'] == day
plt.figure()
plt.plot(power_dt.loc[day_mask, 'Global_active_power'].reset_index(drop=True))
plt.title(f'Global_active_power em {day.date()}')
plt.xlabel('Registros (minutos)'); plt.ylabel('kW'); plt.tight_layout(); plt.show()


### 7) Histograma de `Voltage` e comentários.

In [ ]:

plt.figure()
power['Voltage'].plot(kind='hist', bins=50)
plt.title('Distribuição de Voltage'); plt.xlabel('Volt'); plt.tight_layout(); plt.show()
# Comentário: Em geral, espera-se distribuição aproximadamente unimodal com cauda leve; verificar outliers.


### 8) Consumo médio por mês (todo o período).

In [ ]:

monthly_mean = (power_dt.dropna(subset=['Date_dt'])
                .groupby(power_dt['Date_dt'].dt.to_period('M'))['Global_active_power']
                .mean().to_timestamp())
monthly_mean.head()


### 9) Dia com maior consumo médio de energia ativa global.

In [ ]:

daily_mean_all = (power_dt.dropna(subset=['Date_dt'])
                  .groupby(power_dt['Date_dt'].dt.date)['Global_active_power']
                  .mean())
daily_mean_all.idxmax(), daily_mean_all.max()


### 10) Semana vs Fim de semana – comparação de médias.

In [ ]:

weekday_flag = power_dt['Date_dt'].dt.dayofweek < 5
weekday_mean = power_dt.loc[weekday_flag, 'Global_active_power'].mean()
weekend_mean = power_dt.loc[~weekday_flag, 'Global_active_power'].mean()
weekday_mean, weekend_mean


### 11) Correlação entre variáveis principais.

In [ ]:

cols = ['Global_active_power','Global_reactive_power','Voltage','Global_intensity']
power[cols].corr(method='pearson')


### 12) Nova variável `Total_Sub_metering` = soma das três submedições.

In [ ]:

power_dt['Total_Sub_metering'] = power_dt[['Sub_metering_1','Sub_metering_2','Sub_metering_3']].sum(axis=1)
power_dt[['Total_Sub_metering']].head()


### 13) Mês em que `Total_Sub_metering` > média de `Global_active_power`.

In [ ]:

g_mean = power_dt['Global_active_power'].mean()
monthly_sum_tm = (power_dt
                  .groupby(power_dt['Date_dt'].dt.to_period('M'))['Total_Sub_metering']
                  .mean().to_timestamp())
monthly_sum_tm[monthly_sum_tm > g_mean]


### 14) Série temporal de `Voltage` para 2008.

In [ ]:

mask2008 = power_dt['Date_dt'].dt.year == 2008
plt.figure()
power_dt.loc[mask2008, 'Voltage'].plot()
plt.title('Voltage - 2008'); plt.xlabel('Tempo'); plt.ylabel('Volt'); plt.tight_layout(); plt.show()


### 15) Verão vs Inverno (hemisfério norte).

In [ ]:

# Verão: Jun–Ago; Inverno: Dez–Fev
month = power_dt['Date_dt'].dt.month
summer = power_dt.loc[month.isin([6,7,8]), 'Global_active_power'].mean()
winter = power_dt.loc[month.isin([12,1,2]), 'Global_active_power'].mean()
summer, winter


### 16) Amostragem aleatória 1% e comparação de distribuição.

In [ ]:

sample = power_dt.sample(frac=0.01, random_state=42)
desc_full = power_dt['Global_active_power'].describe()
desc_sample = sample['Global_active_power'].describe()
desc_full, desc_sample


### 17) Normalização Min-Max.

In [ ]:

num_cols = ['Global_active_power','Global_reactive_power','Voltage','Global_intensity']
scaled, scaler = minmax_scale(power_dt.dropna(subset=num_cols), num_cols)
scaled[num_cols].head()


### 18) K-Means (dias → 3 grupos).

In [ ]:

from sklearn.cluster import KMeans
daily_features = (power_dt
                  .dropna(subset=['Date_dt'])
                  .groupby(power_dt['Date_dt'].dt.date)[['Global_active_power','Global_reactive_power','Voltage','Global_intensity']]
                  .mean())
k3 = KMeans(n_clusters=3, n_init=10, random_state=42)
labels = k3.fit_predict(daily_features)
daily_features.assign(cluster=labels).groupby('cluster').mean()


### 19) Decomposição de série temporal (6 meses) – `Global_active_power`.

In [ ]:

from statsmodels.tsa.seasonal import seasonal_decompose
# criar índice datetime completo
pidx = pd.to_datetime(power['Date'] + ' ' + power['Time'], format='%d/%m/%Y %H:%M:%S', errors='coerce')
ts = pd.Series(power['Global_active_power'].values, index=pidx).dropna()
six_months = ts.loc[ts.index.min(): ts.index.min() + pd.DateOffset(months=6)]
res = seasonal_decompose(six_months.asfreq('T').interpolate(), model='additive', period=60*24)  # diário
fig = res.plot(); fig.set_size_inches(8, 6)
plt.tight_layout(); plt.show()


### 20) Regressão linear: prever `Global_active_power` a partir de `Global_intensity`.

In [ ]:

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
df = power_dt.dropna(subset=['Global_active_power','Global_intensity']).sample(100000, random_state=42)  # limitar
X = df[['Global_intensity']].values
y = df['Global_active_power'].values
lr = LinearRegression().fit(X, y)
pred = lr.predict(X)
rmse = mean_squared_error(y, pred, squared=False)
lr.coef_[0], lr.intercept_, rmse


## PARTE 2 – Exercícios adicionais no dataset inicial

### 21) Séries temporais por hora (índice datetime + reamostragem).

In [ ]:

power_idx = add_datetime(power.copy())
g_hourly = power_idx['Global_active_power'].resample('1H').mean()
g_hourly.head(), g_hourly.idxmax(), g_hourly.max()


### 22) Autocorrelação (1h, 24h, 48h). Existem padrões diários?

In [ ]:

def autocorr_at_lag(s, lag):
    return s.autocorr(lag=lag)
ac1 = autocorr_at_lag(g_hourly.dropna(), 1)
ac24 = autocorr_at_lag(g_hourly.dropna(), 24)
ac48 = autocorr_at_lag(g_hourly.dropna(), 48)
ac1, ac24, ac48


### 23) PCA (4 variáveis → 2 componentes).

In [ ]:

from sklearn.decomposition import PCA
subset = power_dt.dropna(subset=['Global_active_power','Global_reactive_power','Voltage','Global_intensity'])
X = subset[['Global_active_power','Global_reactive_power','Voltage','Global_intensity']].values
pca = PCA(n_components=2, random_state=42)
X2 = pca.fit_transform(X)
pca.explained_variance_ratio_, X2[:5]


### 24) PCA + K-Means (3 clusters) – visualização.

In [ ]:

k3_p = KMeans(n_clusters=3, n_init=10, random_state=42)
lab = k3_p.fit_predict(X2)
plt.figure()
plt.scatter(X2[:,0], X2[:,1], s=2, c=lab)
plt.title('PCA (2D) + KMeans (k=3)'); plt.xlabel('PC1'); plt.ylabel('PC2'); plt.tight_layout(); plt.show()
# Pergunta: grupos se separam claramente? Verifique a figura.


### 25) Regressão Linear vs Polinomial (grau 2): `Global_active_power` ~ `Voltage`.

In [ ]:

from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
X = power_dt[['Voltage']].dropna().sample(200000, random_state=42)
y = power_dt.loc[X.index, 'Global_active_power']
# Linear
lin = Pipeline([('lr', LinearRegression())]).fit(X, y)
pred_lin = lin.predict(X)
rmse_lin = mean_squared_error(y, pred_lin, squared=False)
# Polinomial grau 2
poly2 = Pipeline([('poly', PolynomialFeatures(degree=2, include_bias=False)),
                  ('lr', LinearRegression())]).fit(X, y)
pred_poly = poly2.predict(X)
rmse_poly = mean_squared_error(y, pred_poly, squared=False)
rmse_lin, rmse_poly


## PARTE 3 – *Appliances Energy Prediction* (26–35)

### 26) Carregamento e inspeção inicial.

In [ ]:

appl = pd.read_csv(APPLIANCES_PATH)
appl.info(), appl.describe().T.head()


### 27) Distribuição do consumo (hist e série temporal).

In [ ]:

plt.figure(); appl['Appliances'].plot(kind='hist', bins=50); plt.title('Appliances - Hist'); plt.tight_layout(); plt.show()
# série temporal (se houver coluna de tempo 'date')
if 'date' in appl.columns:
    appl['date'] = pd.to_datetime(appl['date'], errors='coerce')
    appl_ts = appl.set_index('date').sort_index()
    plt.figure(); appl_ts['Appliances'].plot(); plt.title('Appliances - Série Temporal'); plt.tight_layout(); plt.show()


### 28) Correlações com variáveis ambientais (temp/umidade).

In [ ]:

env_cols = [c for c in appl.columns if c.lower().startswith(('t','rh'))]  # temperaturas T* e humidades RH*
corrs = appl[['Appliances'] + env_cols].corr().loc['Appliances'].sort_values(ascending=False)
corrs.head(15)


### 29) Normalização Min-Max (reuso posterior).

In [ ]:

num_cols = appl.select_dtypes(include=['int64','float64','int32','float32']).columns.tolist()
appl_scaled, appl_scaler = minmax_scale(appl, num_cols)
appl_scaled[num_cols].head()


### 30) PCA (2 componentes) + plot.

In [ ]:

from sklearn.decomposition import PCA
X_env = appl_scaled[env_cols].dropna().values
pca2 = PCA(n_components=2, random_state=42)
X2_env = pca2.fit_transform(X_env)
pca2.explained_variance_ratio_[:], X2_env[:5]
plt.figure(); plt.scatter(X2_env[:,0], X2_env[:,1], s=2); plt.title('PCA (Ambiente)'); plt.tight_layout(); plt.show()


### 31) Regressão Linear Múltipla para `Appliances`.

In [ ]:

from sklearn.model_selection import train_test_split
X = appl[env_cols].fillna(appl[env_cols].median())
y = appl['Appliances']
Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.2, random_state=42)
lm = LinearRegression().fit(Xtr, ytr)
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
pred = lm.predict(Xte)
r2 = r2_score(yte, pred)
rmse = mean_squared_error(yte, pred, squared=False)
mae = mean_absolute_error(yte, pred)
r2, rmse, mae


### 32) Random Forest Regressor – comparação com Linear.

In [ ]:

from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=200, random_state=42, n_jobs=-1)
rf.fit(Xtr, ytr)
pred_rf = rf.predict(Xte)
r2_rf = r2_score(yte, pred_rf)
rmse_rf = mean_squared_error(yte, pred_rf, squared=False)
mae_rf = mean_absolute_error(yte, pred_rf)
(r2, rmse, mae), (r2_rf, rmse_rf, mae_rf)


### 33) K-Means (3–5 clusters) – perfis de consumo.

In [ ]:

from sklearn.cluster import KMeans
for k in [3,4,5]:
    km = KMeans(n_clusters=k, n_init=10, random_state=42)
    labs = km.fit_predict(X)
    print(f'k={k} | inertia={km.inertia_:.2f}')


### 34) Classificação binária: alto vs baixo consumo (mediana).

In [ ]:

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
y_bin = (appl['Appliances'] > appl['Appliances'].median()).astype(int)
Xtr, Xte, ytr, yte = train_test_split(X, y_bin, test_size=0.2, random_state=42)
logit = LogisticRegression(max_iter=1000).fit(Xtr, ytr)
rf_clf = RandomForestClassifier(n_estimators=300, random_state=42, n_jobs=-1).fit(Xtr, ytr)
pred_l = logit.predict(Xte)
pred_r = rf_clf.predict(Xte)
pred_proba_r = rf_clf.predict_proba(Xte)[:,1]


### 35) Avaliação (matriz de confusão e métricas). O modelo erra mais alto ou baixo?

In [ ]:

from sklearn.metrics import confusion_matrix, classification_report
cm_l = confusion_matrix(yte, pred_l)
cm_r = confusion_matrix(yte, pred_r)
print('Logistic Regression\n', cm_l)
print(classification_report(yte, pred_l))
print('Random Forest\n', cm_r)
print(classification_report(yte, pred_r))
# Compare as taxas de FN/FP para inferir onde o erro é maior (alto vs baixo consumo).


## PARTE 4 – Orange Data Mining (36–40) – Guia Rápido


**36)** CSV File Import → Data Table. *Pergunta*: quantas variáveis/registros?  
**37)** Sample Data (1%). Compare distribuição com a base completa (Distribution/Box Plot).  
**38)** Distribution → `Global_active_power`: verifique concentração em valores baixos vs altos.  
**39)** Scatter Plot: `Voltage` (X) × `Global_intensity` (Y). Há correlação visível?  
**40)** k-Means (k=3) usando `Sub_metering_1..3`; visualize no Scatter Plot e descreva os perfis.

> Dica: use Color = cluster no Scatter Plot; *Silhouette Score* do widget ajuda a avaliar separação.
